# Release 100 Prisoners

## Problem

1. 100명의 죄수들은 1 ~ 100까지의 번호를 가지고 있다.
2. 1 ~ 100까지 적힌 종이가 방 안에 들어있는 100개의 번호가 적힌 상자 안에 랜덤하게 들어가 있다.
3. 각 죄수는 방에 들어가 50개의 상자를 열어 볼 수 있다.
4. 50개를 열어 본 뒤 즉시 떠나야 하며 어떠한 방식으로도 추가적인 소통은 불가능하다.
5. 100명의 죄수들이 모두 자신의 번호를 찾는다면 석방이며 한 명이라도 번호를 찾지 못하면 모두 처형된다.
6. 첫 번째 죄수가 방에 들어가기 전까지 전략 회의를 할 수 있다.

## Simple Strategy

각 죄수들이 100의 상자 중에서 50개의 상자를 열어 자신의 번호를 찾을 확률은 $\dfrac{50}{100} = \dfrac{1}{2}$이다. 만약 죄수들이 무작위로 자신의 번호를 탐색한다면 100명의 죄수들이 자신들이 번호를 모두 찾을 확률은 다음과 같다:

$$
\dfrac{1}{2} \times \dfrac{1}{2} \times \cdots \dfrac{1}{2} = (\dfrac{1}{2})^{100}
$$

In [1]:
p = (1 / 2) ** 100
print(f"죄수 번호를 모두 찾을 확률: {p}")

죄수 번호를 모두 찾을 확률: 7.888609052210118e-31


이 확률은 그냥 불가능이다!!! 100명 모두가 어떻게 자신의 번호를 찾는다는 말인가? 

앞서 [Problem](#problem)의 6에 의하면 첫 번째 죄수가 방에 들어가기 전까지 전략 회의가 가능하다고 했다. 이에 따라 어떠한 전략을 세워서 확률을 끌어올릴 방법이 있을까? 몰론 있을 것이다. 그런데 $(\dfrac{1}{2})^{100}$의 확률을 무려 $0.3$ 정도 까지 끌어올릴 수 있는 굉장히 단순한 전략이 있다고 하면 믿을 수 있는가? 자 이제 알아보자.

## Loop Strategy

핵심 아이디어는 이렇다:

1. 자신의 번호가 적힌 상자를 연다.
2. 상자 속에 있는 종이의 번호가 적힌 상자를 연다.
3. 2 과정을 반복하면 횟수 제한이 없다고 할 때 반드시 자신의 번호를 발견하게 된다.

이게 도대체 무슨 소리일까? 먼저 예시를 들어보자. 죄수 번호가 32라고 하자. 32가 적힌 상자에는 57이라고 적힌 종이가 있다. 57이 적힌 상자에는 17이라고 적힌 종이가 있다. 17이 적힌 상자에는 32가 적힌 종이가 있다. 32가 적힌 상자에는 57이라고 적힌 종이가 있다. 이 과정은 무한 루프이다.

즉, 이것이 loop strategy이다. 여기서 중요한 사실은 어떤 번호가 적힌 상자는 반드시 어떤 종이가 가리키고 있다는 사실이다. 내가 $a$라고 적힌 상자를 열어 $b$라는 종이를 발견하여 $b$라는 상자를 다시 연다. 즉, $b$라고 적힌 상자는 $a$ 상자 내에 있는 $b$라는 종이가 가리키고 있었음이 보장된다. 그렇다면 $a$는? 역시 어떤 상자 속의 종이가 가리킬 것이다. 따라서 이런 전략으로 따라가다보면 언젠가는 $a$에 도달하게 된다. 따라서 loop를 형성한다.

위 아이디어를 바탕으로 자신의 번호를 반드시 발견하고 싶다면 어떻게 해야할까? 맨 처음에 자신의 번호가 적힌 상자를 열어야만 한다. 자신의 번호가 적힌 상자는 이 상자가 포함된 loop 속에서 어떤 종이가 분명히 가리키고 있을 것이기 때문이다. 즉, loop의 길이가 $L$이라면 $L$번째 상자에 맨 첫번째 상자의 번호가 적힌 종이가 있다.

아직 직관적이지 않은가? 시뮬레이션을 해보자!

### 100개의 상자에 랜덤한 종이를 삽입하는 함수

In [2]:
import numpy as np
np.random.seed(2)

def make_random_box(num_boxes: int) -> np.ndarray:
    return np.random.permutation(num_boxes) + 1

In [3]:
random_box = make_random_box(100)
print(random_box.reshape(10, 10)) # for beautiful display

[[ 84  31  57  25  17  24   3  28  29  14]
 [100  93  77  15   1  22   4  30  62  80]
 [ 36  12  85  45  74   6  26  78  75  63]
 [ 66   2  19  49  37  79   7  90  92  11]
 [ 13  54  88  55  96  33  20  27  61  56]
 [ 10  97  18  60  58  42  65  46  98   9]
 [ 72  95  91  99  87  81  51  53  67  89]
 [ 71  47  69  70  82  59  34  39  52  43]
 [  5  68  40  38  21  32  64  48  86  94]
 [ 50  35   8  76  83  44  23  73  16  41]]


### Loop Strategy 시뮬레이션

In [13]:
prisoner_numbers = [99, 28, 73, 32]

for prisoner_number in prisoner_numbers:
    box_number = prisoner_number # initialize number of prisoner
    box_numbers = [box_number]
    is_discovered = False
    count = 0

    while not is_discovered:
        count += 1
        box_number = random_box[box_number - 1]
        box_numbers.append(box_number)
        is_discovered = box_number == prisoner_number
        
    print(f"Prisoner Number: {prisoner_number}, Loop length: {len(box_numbers) - 1}")
    print(box_numbers)
    print()

Prisoner Number: 99, Loop length: 18
[99, 16, 22, 12, 93, 8, 28, 78, 39, 92, 35, 37, 7, 3, 57, 65, 87, 64, 99]

Prisoner Number: 28, Loop length: 18
[28, 78, 39, 92, 35, 37, 7, 3, 57, 65, 87, 64, 99, 16, 22, 12, 93, 8, 28]

Prisoner Number: 73, Loop length: 15
[73, 69, 67, 51, 10, 14, 15, 1, 84, 38, 90, 94, 76, 59, 98, 73]

Prisoner Number: 32, Loop length: 13
[32, 2, 31, 66, 81, 5, 17, 4, 25, 74, 70, 89, 86, 32]



와우! `prison_number`가 어떤 값이든 루프를 형성함을 알 수 있다. 또한 `prison_number` 값에 따라 다른 종류의 loop가 형성될 수 있다는 사실도 보인다. 구체적으로 `99`와 `28`은 시작점만 다를 뿐, 동일 원소로 구성되어 있기 때문에 같은 loop이다. 반대로 `99`, `73`, `32`는 완전히 서로 다른 종류의 loop이다. 참고로 죄수 번호 `99`에 대해 loop 구성 요소는 `99` ~ `64`까지이기 때문에 마지막 원소 `99`는 제외한다. 루프가 형성되고 있음을 보이기 위해 포함한 것 뿐이다.

곧 내용 추가될 예정...